In [1]:
#Setup
import collections
import pandas as pd
import numpy as np
import spacy
import pickle
from spacy.tokens import DocBin
from spacy.tokens import Doc
import datetime
import dateparser
import seaborn as sns
import yfinance as yf

nlp = spacy.load('en_core_web_md')

import matplotlib.pyplot as plt
import squarify
import seaborn as sns
plt.rcParams["figure.figsize"] = [20,10]

#Set sentiment extensions
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()
def sentiment_scores(docx):
    return sent_analyzer.polarity_scores(docx.text)
Doc.set_extension("sentimenter",getter=sentiment_scores,force=True)


In [213]:
#Trump became the president as of January 20, 2017, but here we extract all the tweets as of 2007.
#Also we need to remove retweets
df1 = pd.read_excel('tweets.xls', parse_dates=['created_at'])
df1 = df1[df1['created_at'] > dateparser.parse('01/01/2017 UTC')]
df1 = df1[df1['is_retweet'] == False]
df1 = df1.drop(['id_str', 'is_retweet'], axis=1)

In [214]:
#Remove trivial tweets
#For now I just removed all the tweets with a length of smaller than 5.
df1['len'] = df1['text'].apply(lambda x: len(str(x).split(" ")))
df1 = df1[df1['len']>5]

In [215]:
#Aggregate the tweets by day
df2 = df1.groupby(df1['created_at'].dt.date)['text'].apply(lambda x:' '.join(x))
df2 = pd.DataFrame(df2)
df2.index = pd.to_datetime(df2.index, format='%Y-%m-%d')
df2['num'] = df1.groupby(df1['created_at'].dt.date)['text'].count()

In [216]:
#Process the stock price file
Y_day = pd.read_excel('S&P 500.xls')
Y_day['is_up'] = (Y_day['Close'].shift(-1) > Y_day['Close']) * 1
Y_day['diff'] = Y_day['Close'].shift(-1) - Y_day['Close']
Y_day = Y_day[['Date', 'is_up', 'diff']]

In [217]:
#Join both dataframe together
df = df2.merge(Y_day, left_on='created_at', right_on='Date')
df.columns = ['text', 'num', 'date', 'is_up', 'diff']

In [218]:
#Save the dataframe
df_temp = df
df_temp['date'] = df_temp['date'].astype(str)
df_temp.to_excel('tweets_new.xls', index=False)

In [186]:
#Apply NLP models on the remaining tweets
def nlp_apply(file_from, file_to):
    df = pd.read_excel(file_from)
    tweets = df['text']

    doc_bin = DocBin(attrs=['LEMMA', 'ENT_IOB', 'ENT_TYPE', 'is_digit', 'like_url', 'like_num', 
                            'DEP', "POS", 'TAG', 'SHAPE', 'is_alpha', 'is_stop', 'is_oov'], store_user_data=True)

    for doc in nlp.pipe(tweets):
        doc_bin.add(doc)
    bytes_data = doc_bin.to_bytes()

    #Save to a pickle file
    pickle.dump(bytes_data, open(file_to, 'wb'))

In [191]:
#A help function to read the pickle file
def read_data(path):
    tweets = pickle.load(open(path, 'rb'))
    doc_bin = DocBin().from_bytes(tweets)
    docs = list(doc_bin.get_docs(nlp.vocab))
    return docs
tweets = read_data('tweets_new.p')

In [263]:
#Split the dataset into a training and holdout set(it is for the final validation, not for the CV!)
#The holdout set begins as of September of 2019

df_train = df.iloc[:657]
df_test = df.iloc[657:]
df_train.to_excel('tweets_train.xls', index=False)
df_test.to_excel('tweets_test.xls', index=False)
nlp_apply('tweets_train.xls', 'tweets_train.p')
nlp_apply('tweets_test.xls', 'tweets_test.p')

# Experiment for the hourly data extraction

In [114]:
#Trump became the president as of January 20, 2017, but here we extract all the tweets as of 2007.
#Also we need to remove retweets
df1 = pd.read_excel('tweets.xls', parse_dates=['created_at'])
df1 = df1[df1['created_at'] > dateparser.parse('01/01/2017 UTC')]
df1 = df1[df1['is_retweet'] == False]
df1 = df1.drop(['id_str', 'is_retweet'], axis=1)

In [115]:
#Remove trivial tweets
#For now I just removed all the tweets with a length of smaller than 5.
df1['len'] = df1['text'].apply(lambda x: len(str(x).split(" ")))
df1 = df1[df1['len']>5]

In [116]:
#Aggregate the tweets by hour
df1 = df1.set_index(['created_at'])
df1 = df1.resample('60Min')['text'].apply(lambda x:' '.join(x))
df2 = pd.DataFrame(df1)
df2 = df2[df2['text']!='']

In [50]:
#Process the stock price file
Y_hour = yf.download("^GSPC", start="2018-01-01", end="2019-11-25", interval='60m')
Y_hour.index = Y_hour.index.tz_convert('UTC')

[*********************100%***********************]  1 of 1 completed


In [132]:
#Join both dataframe together
df2['get_close_price'] = df2.index + pd.Timedelta(minutes=30)
df2['get_open_price'] = df2.index - pd.Timedelta(minutes=30)
df2 = df2.reset_index()
df2 = df2.merge(Y_hour['Open'], left_on='get_open_price', right_on=Y_hour.index, how='left')
df2 = df2.merge(Y_hour['Close'], left_on='get_close_price', right_on=Y_hour.index, how='left')
df2 = df2.set_index('created_at')

In [134]:
#Drop columns without price data
df2 = df2.dropna()

In [136]:
#Make labels
df2['is_up'] = (df2['Close'] > df2['Open']) * 1
df2['diff'] = df2['Close'] - df2['Open']

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [187]:
#Split the dataset into a training and holdout set(it is for the final validation, not for the CV!)
#The holdout set begins as of September of 2019
df = df2.tz_localize(None)
df['get_close_price'] = df['get_close_price'].astype(str)
df['get_open_price'] = df['get_open_price'].astype(str)

df_train = df.iloc[:638]
df_test = df.iloc[638:]
df_train.to_excel('tweets_train_hourly.xls', index=False)
df_test.to_excel('tweets_test_hourly.xls', index=False)
nlp_apply('tweets_train_hourly.xls', 'tweets_train_hourly.p')
nlp_apply('tweets_test_hourly.xls', 'tweets_test_hourly.p')